# 基于MindSpore实现Q-Learning算法的示例
   
   本实验介绍基于MindSpore实现Q-Learning的示例，参考作者mofan用到的一个名为“treasure on right”的简单游戏来实现Q-Learning这个算法。

# 1. 实验目的
- 理解Q-Learning算法原理。
- 理解动作的奖励R和状态的转移S的工作流程。
- 自定义函数用于训练智能体。

# 2.  算法知识点介绍


## 2.1 Q-Learning算法
- Q-Learning算法是一种基于值迭代的强化学习算法，用于解决马尔可夫决策过程（MDP）问题。
- Q-Learning算法的目标是通过迭代更新Q值，使得智能体能够学习到最优的策略。在训练过程中，Q值会逐渐收敛到最优值，从而使智能体能够在不同状态下选择最佳动作。
- 需要注意的是，Q-Learning算法是一种基于离散状态和离散动作的算法。对于连续状态和动作空间，可以使用函数逼近方法（如神经网络）来近似Q值函数。此外，Q-Learning算法也可以与其他技术（如经验回放、深度Q网络等）结合使用以提高训练效果和稳定性。

## 2.2 Q-Learning算法的基本步骤：
- 初始化Q表：创建一个Q表，其中行表示状态，列表示动作。初始时，Q表中的所有元素都为0或随机值。
- 选择动作：根据当前状态和Q表选择动作。通常使用ε-greedy策略，以一定的概率选择随机动作（探索），以较高概率选择具有最大Q值的动作（利用）。
- 执行动作并观察环境：根据选择的动作与环境进行交互，获得下一个状态和即时奖励。
- 更新Q值：使用Q-Learning更新规则更新Q表中的Q值。根据当前状态、执行的动作、下一个状态和即时奖励，更新对应的Q值。
- 更新状态：将下一个状态设置为当前状态，重复步骤2-4，直到达到终止状态或达到最大训练步数。
- 循环训练：重复多个训练回合，直到Q值收敛或达到最大训练回合数。

## 2.3 gym平台介绍
本实验借助了gym平台的环境，该平台由 openai 公司开发，且提供了一整套与平台中虚拟环境进行交互的 api接口，gym 的推出为强化学习算法的研究提供了更好地基准测试平台，同时将各类 环境标准化，使研究员可以专注于算法研究而无需花过多的时间在环境的模拟上。gym 提供一个 step 函数供智能体与环境进行交互，其参数为动作，主要返回值及含义分别为：
- state：表示智能体所处环境的当前状态，代表着智能体的观察值即状态。
- reward：表示智能体采取操作后从环境中获得的奖励，其类型可能是整数、小数等，但是具体的规模和类型与具体的规模和类型与环境有关，但是智能体的总目标仍然是获取最大的奖励值。
- done: 大多数任务都属于阶段性任务，当到达一定条件的时候表示任务已经结束，比如五子棋游戏中的一方五子相连，机器人在路面上摔倒，或者在规定的步数以内没有完成任务，则都代表任务结束。所以 done 是一个判断条件，类型为布尔值，代表当前任务是否结束，如果结束则可以选择使用 reset 函数重置当前任务。

# 3. 实验环境
在动手进行实践之前，需要注意以下几点：
* 确保实验环境正确安装，包括安装MindSpore。安装过程：首先登录[MindSpore官网安装页面](https://www.mindspore.cn/install)，根据安装指南下载安装包及查询相关文档。同时，官网环境安装也可以按下表说明找到对应环境搭建文档链接，根据环境搭建手册配置对应的实验环境。
* 推荐使用交互式的计算环境Jupyter Notebook，其交互性强，易于可视化，适合频繁修改的数据分析实验环境。
* 实验也可以在华为云一站式的AI开发平台ModelArts上完成。
* 推荐实验环境：MindSpore版本=2.0；Python环境=3.7


|  硬件平台 |  操作系统  | 软件环境 | 开发环境 | 环境搭建链接 |
| :-----:| :----: | :----: |:----:   |:----:   |
| CPU | Windows-x64 | MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第二章2.1节和第三章3.1节](./MindSpore环境搭建实验手册.docx)|
| GPU CUDA 10.1|Linux-x86_64| MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第二章2.2节和第三章3.1节](./MindSpore环境搭建实验手册.docx)|
| Ascend 910  | Linux-x86_64| MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第四章](./MindSpore环境搭建实验手册.docx)|

# 4. 数据处理

## 4.1 实验准备

In [39]:
N_STATES = 6                      # 定义的环境下，有6个状态
ACTIONS = ["left", "right"]       # 有两个动作
EPSILON = 0.9                     # ε-greedy算法中的探索率
ALPHA = 0.1                       # 学习率
GAMMA = 0.9                       # 折扣因子
MAX_EPISODES = 15                 # 最大训练回合数
FRESH_TIME = 0.3                  # 刷新环境的时间间隔
TerminalFlag = "terminal"         # 有一个终止状态

## 4.2 数据加载

如果没有安装`gym`，请使用以下命令安装：

pip install gym

In [40]:
# gym强化学习库, gym版本为0.26.2
import gym
# 加载车杆游戏场景
env=gym.make('CartPole-v1')
# 获取状态数
state_number=env.observation_space.shape[0]
# 获取动作数
action_number=env.action_space.n

# 5. 算法实现

使用MindSpore实现Q-Learning算法并进行训练。

## 5.1 导入Python库并配置运行信息

In [41]:
import time
import numpy as np
import pandas as pd
import mindspore as ms

## 5.2 设置实验环境和参数

In [42]:
# 设置计算环境
ms.set_context(mode=ms.GRAPH_MODE, device_target="CPU")

N_STATES = 6                      # 定义的环境下，有6个状态
ACTIONS = ["left", "right"]       # 有两个动作
EPSILON = 0.9                     # ε-greedy算法中的探索率
ALPHA = 0.1                       # 学习率
GAMMA = 0.9                       # 折扣因子
MAX_EPISODES = 15                 # 最大训练回合数
FRESH_TIME = 0.3                  # 刷新环境的时间间隔
TerminalFlag = "terminal"         # 有一个终止状态

## 5.3 建立Q表

`build_q_table` 函数用于创建一个 Q 表，它使用 pandas 的 DataFrame 表示。Q 表中的所有值都初始化为零。

In [43]:
def build_q_table(n_states, actions):
    return pd.DataFrame(
        np.zeros((n_states, len(actions))),
        columns=actions
    )

## 5.4 根据当前状态和Q表选择动作

`choose_action` 函数根据当前状态和 Q 表选择一个动作。该函数接受两个参数：
- state：  当前状态。
- q_table：  Q 表，使用 pandas DataFrame 表示。

在函数中，首先从 Q 表中获取当前状态对应的行 state_table。然后，通过以下两个条件来决定选择的动作：
- 1、如果随机生成的均匀分布值大于 EPSILON，或者 state_table 中所有的值都为零，则以一定概率随机选择一个动作。
- 2、否则，选择 state_table 中具有最大 Q 值的动作作为选择。

最后，函数返回选定的动作名称。

In [44]:
def choose_action(state, q_table):
    state_table = q_table.loc[state, :]
    if (np.random.uniform() > EPSILON) or ((state_table == 0).all()):
        action_name = np.random.choice(ACTIONS)  # 以一定概率随机选择动作
    else:
        action_name = state_table.idxmax()       # 选择Q值最大的动作
    return action_name

## 5.5 获取环境反馈

使用`get_env_feedback`函数用于获取在给定当前状态和选择的动作下的环境反馈。该函数接受两个参数：
- S：当前状态。
- A：选择的动作

在函数中，根据动作 A 和当前状态 S 的取值，计算下一个状态 S_ 和相应的奖励 R。具体地：

1. 如果动作 A 是 "right"，则：
- 如果当前状态 S 是倒数第二个状态（即达到终止状态前的最后一个状态），则下一个状态 S_ 设置为终止标志 TerminalFlag，奖励 R 设置为 1（到达终止状态的奖励）。
- 否则，下一个状态 S_ 设置为当前状态 S 加 1（向右移动一个状态），奖励 R 设置为 0（向右移动的奖励）。

2. 如果动作 A 是 "left"，则下一个状态 S_ 设置为当前状态 S 减 1（向左移动一个状态），奖励 R 设置为 0（向左移动的奖励）。

3. 最后，函数返回下一个状态 S_ 和奖励 R。

In [45]:
def get_env_feedback(S, A):
    if A == "right":
        if S == N_STATES - 2:
            S_, R = TerminalFlag, 1  # 到达终止状态的奖励为1
        else:
            S_, R = S + 1, 0         # 向右移动的奖励为0
    else:
        S_, R = max(0, S - 1), 0    # 向左移动的奖励为0
    return S_, R

## 5.6 更新环境

使用`update_env` 函数用于更新环境并显示当前状态和可选的动作。该函数接受三个参数：
- S：当前状态。
- episode：当前回合的索引。
- step_counter：当前回合中的步数计数器。


In [46]:
# 用于更新环境，展示当前状态和可选的动作
def update_env(S, episode, step_counter): 
    env_list = ["-"] * (N_STATES - 1) + ["T"]
    if S == TerminalFlag:
        interaction = 'Episode %s: total_steps = %s' % (episode + 1, step_counter)
        print(interaction)
        time.sleep(2)
    else:
        env_list[S] = '0'
        interaction = ''.join(env_list)
        print(interaction)
        time.sleep(FRESH_TIME)

在函数中，根据当前状态 S 更新环境的显示。具体地：

- 如果当前状态 S 等于终止标志 TerminalFlag，则打印出当前回合的索引和总步数，并暂停程序执行 2 秒。
- 否则，根据当前状态 S 将环境列表 env_list 中对应位置的字符更新为 '0'，表示当前位置。然后将环境列表转换为字符串 interaction。
- 最后，打印出环境的字符串表示并暂停程序执行 FRESH_TIME 秒。

这样，函数实现了根据当前状态更新环境并显示当前状态和可选的动作。在终止状态下，会打印出当前回合的索引和总步数作为回合结束的提示。

# 6. 模型训练

通过与环境的交互，不断更新 Q 表，使得智能体能够学习到最优的策略。

In [47]:
def rl():
    q_table = build_q_table(N_STATES, ACTIONS)  # 建立初始的 Q 表
    for episode in range(MAX_EPISODES):  # 进行多个回合的训练
        step_counter = 0  # 记录每个回合的步数
        S = 0  # 初始状态为 0
        is_terminated = False  # 判断是否终止的标志位
        update_env(S, episode, step_counter)  # 更新环境显示
        while not is_terminated:
            A = choose_action(S, q_table)  # 根据当前状态和 Q 表选择动作
            S_, R = get_env_feedback(S, A)  # 获取环境反馈，得到下一个状态和奖励值
            q_predict = q_table.loc[S, A]  # 获取当前状态和选择的动作的 Q 值

            if S_ != TerminalFlag:  # 如果下一个状态不是终止状态
                q_target = R + GAMMA * q_table.loc[S_, :].max()  # 计算目标 Q 值，采用贝尔曼方程更新
            else:
                q_target = R  # 如果下一个状态是终止状态，则目标 Q 值为当前的奖励值，没有后续的动作选择
                is_terminated = True  # 设置终止标志位为 True
            q_table.loc[S, A] += ALPHA * (q_target - q_predict)  # 更新 Q 表
            S = S_  # 更新当前状态为下一个状态
            update_env(S, episode, step_counter + 1)  # 更新环境显示
            step_counter += 1  # 步数加1
    return q_table

# 7. 模型预测

打印出Q表的结果。

In [48]:
if __name__ == '__main__':
    q_table = rl()
    print(q_table)

0----T
0----T
-0---T
--0--T
-0---T
0----T
-0---T
0----T
0----T
-0---T
0----T
-0---T
--0--T
-0---T
0----T
-0---T
0----T
-0---T
0----T
0----T
-0---T
--0--T
-0---T
0----T
0----T
0----T
0----T
-0---T
0----T
0----T
0----T
-0---T
--0--T
---0-T
----0T
---0-T
--0--T
---0-T
----0T
---0-T
--0--T
---0-T
--0--T
-0---T
--0--T
---0-T
--0--T
-0---T
0----T
-0---T
--0--T
-0---T
0----T
-0---T
0----T
-0---T
0----T
0----T
-0---T
0----T
-0---T
0----T
0----T
-0---T
0----T
-0---T
--0--T
-0---T
0----T
0----T
0----T
0----T
-0---T
0----T
0----T
-0---T
0----T
0----T
0----T
-0---T
0----T
0----T
0----T
0----T
-0---T
0----T
-0---T
0----T
-0---T
0----T
0----T
0----T
-0---T
0----T
-0---T
0----T
0----T
0----T
0----T
-0---T
0----T
-0---T
0----T
-0---T
0----T
-0---T
0----T
-0---T
--0--T
-0---T
--0--T
-0---T
--0--T
-0---T
--0--T
---0-T
--0--T
-0---T
0----T
0----T
-0---T
--0--T
---0-T
--0--T
-0---T
--0--T
---0-T
----0T
---0-T
--0--T
---0-T
--0--T
-0---T
--0--T
---0-T
----0T
Episode 1: total_steps = 136
0----T
0----T
-0---